<a href="https://colab.research.google.com/github/yalopez84/Interpretable_method/blob/master/Triple_Prediction_FB_KG_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
!pip install pytorch-pretrained-bert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [56]:
from __future__ import absolute_import, division, print_function
import argparse
import csv
import logging
from datetime import datetime
import os
import random
import sys
from shutil import rmtree
import pdb
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange
from torch.nn import CrossEntropyLoss, MSELoss
from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score
from sklearn import metrics
from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

logger = logging.getLogger(__name__)

In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
data_dir="/content/drive/MyDrive/Almacen_phd/FB13/"
os.chdir(data_dir)

# Preprocessing-Start

In [59]:
class InputExample(object):

    def __init__(self, guid, text_a, text_b=None, text_c=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.text_c = text_c
        self.label = label

In [60]:
class Triple(object):
    def __init__(self, guid, subject , predicate , obj, label):
        self.guid=guid
        self.subject=subject
        self.predicate=predicate
        self.obj=obj
        self.label=label

In [61]:
class InputFeatures(object):
    
    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

In [62]:
class TripleEvaluationReport(object):
    
    def __init__(self, id_triple, id_label, id_prediction):
        self.id_triple = id_triple
        self.id_label = id_label
        self.id_prediction = id_prediction

In [63]:
class DataProcessor(object):
  
    def get_train_examples(self, data_dir):
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        raise NotImplementedError()

    def get_labels(self, data_dir):
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines

In [64]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()
    
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train_reduced.tsv")), "train", data_dir)

    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev", data_dir)

    def get_test_examples(self, data_dir):
        return self._create_examples(
          self._read_tsv(os.path.join(data_dir, "test_reduced.tsv")), "test", data_dir)

    def get_relations(self, data_dir):
        with open(os.path.join(data_dir, "relations.txt"), 'r') as f:
            lines = f.readlines()
            relations = []
            for line in lines:
                relations.append(line.strip())
        return relations

    def get_labels(self, data_dir):
        return ["0", "1"]

    def get_entities(self, data_dir):
        with open(os.path.join(data_dir, "entities.txt"), 'r') as f:
            lines = f.readlines()
            entities = []
            for line in lines:
                entities.append(line.strip())
        return entities

    def get_train_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "train.tsv"))

    def get_dev_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "dev.tsv"))

    def get_test_triples(self, data_dir):
        return self._read_tsv(os.path.join(data_dir, "test.tsv"))

    def _create_examples(self, lines, set_type, data_dir):

        train_with_corrupts=[] 
        id_Triple=""
        subject=""
        predicate=""
        obj=""
        id_Triple_corrupt=""
        subject_Triple_corrupt=""
        predicate_Triple_corrupt=""
        obj_Triple_corrupt=""

        ent2text = {}
        with open(os.path.join(data_dir, "entity2text.txt"), 'r') as f:
            ent_lines = f.readlines()
            for line in tqdm(ent_lines):
                temp = line.strip().split('\t')
                if len(temp) == 2:
                    ent2text[temp[0]] = temp[1]

        entities = list(ent2text.keys())

        rel2text = {}
        with open(os.path.join(data_dir, "relation2text.txt"), 'r') as f:
            rel_lines = f.readlines()
            for line in rel_lines:
                temp = line.strip().split('\t')
                rel2text[temp[0]] = temp[1]      

        lines_str_set = set(['\t'.join(line) for line in lines])
        examples = []        

        for (i, line) in enumerate(lines):
            auxiliar=["loke_joseph_of_austria_palatine_of_hungary","united_srena_gale"]
            flag_aux=False

            if line[0] or line[2] not in auxiliar:
                head_ent_text = ent2text[line[0]]
                tail_ent_text = ent2text[line[2]]
                relation_text = rel2text[line[1]]          
                flag_aux=True
            else:
                head_ent_text = line[0]
                tail_ent_text = line[2]
                relation_text = line[1]
                flag_aux=True

            if set_type == "dev" or set_type == "test" and flag_aux:
                triple_label = line[3]
                if triple_label == "1":
                    label = "1"
                else:
                    label = "0"

                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 
                self.labels.add(label)
                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label=label))
                
            elif set_type == "train" and flag_aux:
                guid = "%s-%s" % (set_type, i)
                text_a = head_ent_text
                text_b = relation_text
                text_c = tail_ent_text 

                id_Triple=guid
                subject=line[0]
                predicate=line[1]
                obj=line[2]
                train_with_corrupts.append(Triple(guid=id_Triple,subject= subject, predicate=predicate, obj=obj, label="1"))

                examples.append(
                    InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = text_c, label="1"))

                rnd = random.random()
                guid = "%s-%s" % (set_type + "_corrupt", i)
                if rnd <= 0.5:
                    # corrupting head
                    tmp_head = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[0])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_head = random.choice(tmp_ent_list)
                        tmp_triple_str = tmp_head + '\t' + line[1] + '\t' + line[2]
                        if tmp_triple_str not in lines_str_set:
                            break                    
                    tmp_head_text = ent2text[tmp_head]

                    id_Triple_corrupt=guid
                    subject_Triple_corrupt=tmp_head
                    predicate_Triple_corrupt=line[1]
                    obj_Triple_corrupt=line[2]
                    train_with_corrupts.append(Triple(guid=id_Triple_corrupt,subject=subject_Triple_corrupt, predicate=predicate_Triple_corrupt, obj=obj_Triple_corrupt, label="0"))
                    
                    examples.append(
                        InputExample(guid=guid, text_a=tmp_head_text, text_b=text_b, text_c = text_c, label="0"))       
                else:
                    # corrupting tail
                    tmp_tail = ''
                    while True:
                        tmp_ent_list = set(entities)
                        tmp_ent_list.remove(line[2])
                        tmp_ent_list = list(tmp_ent_list)
                        tmp_tail = random.choice(tmp_ent_list)
                        tmp_triple_str = line[0] + '\t' + line[1] + '\t' + tmp_tail
                        if tmp_triple_str not in lines_str_set:
                            break
                    tmp_tail_text = ent2text[tmp_tail]

                    id_Triple_corrupt=guid
                    subject_Triple_corrupt=line[0]
                    predicate_Triple_corrupt=line[1]
                    obj_Triple_corrupt=tmp_tail
                    train_with_corrupts.append(Triple(guid=id_Triple_corrupt,subject= subject_Triple_corrupt, predicate=predicate_Triple_corrupt, obj=obj_Triple_corrupt, label="0"))
                    
                    examples.append(
                        InputExample(guid=guid, text_a=text_a, text_b=text_b, text_c = tmp_tail_text, label="0"))     
        
        if set_type == "train":
            train_with_negatives_file=os.path.join(data_dir, "train_reduced_with_negatives.txt")
            with open(train_with_negatives_file, "w") as writer:
                for triple in train_with_corrupts:
                    writer.write("%s %s %s %s %s\n" % (triple.guid, triple.subject, triple.predicate, triple.obj, triple.label))
                                                            
        return examples

In [65]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, print_info = True):
    
    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    logger.info("")
    logger.info("Writing examples converted to features")
    logger.info("max_seq_length %d" % (max_seq_length))

    for (ex_index, example) in enumerate(examples):
        
        tokens_a = tokenizer.tokenize(example.text_a)
        tokens_b = None
        tokens_c = None

        if example.text_b and example.text_c:
            tokens_b = tokenizer.tokenize(example.text_b)
            tokens_c = tokenizer.tokenize(example.text_c)
            _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_seq_length - 4)
        else:
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)
        if tokens_c:
            tokens += tokens_c + ["[SEP]"]
            segment_ids += [0] * (len(tokens_c) + 1)        

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        input_mask = [1] * len(input_ids)

        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        label_id = label_map[example.label]

        if ex_index < 2 and print_info:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

In [66]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

In [67]:
def _truncate_seq_triple(tokens_a, tokens_b, tokens_c, max_length):
    while True:
        total_length = len(tokens_a) + len(tokens_b) + len(tokens_c)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b) and len(tokens_a) > len(tokens_c):
            tokens_a.pop()
        elif len(tokens_b) > len(tokens_a) and len(tokens_b) > len(tokens_c):
            tokens_b.pop()
        elif len(tokens_c) > len(tokens_a) and len(tokens_c) > len(tokens_b):
            tokens_c.pop()
        else:
            tokens_c.pop()

In [68]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

In [69]:
def compute_metrics(task_name, preds, labels):
    
    assert len(preds) == len(labels)
    if task_name == "kg":
        return {"acc": simple_accuracy(preds, labels)}
    else:
        raise KeyError(task_name)

In [70]:
def createFBReduced(test_start,test_end,test_step,train_start,train_end,train_step):
    #To reduce FB dataset 
    #train.tsv has 316219 triples, train_reduced.tsv has a subset of 11082 triples
    #test.tsv has 47464, test_reduced has a sub set of 2253 triples  

    processor = KGProcessor()
    test_lines=processor._read_tsv(os.path.join(data_dir, "test.tsv"))
    train_lines=processor._read_tsv(os.path.join(data_dir, "train.tsv")) 

    triples_train_reduced=processor._read_tsv(os.path.join(data_dir, "train_reduced.tsv"))
    triples_test_reduced=processor._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))
    flag=False

    for test_line in tqdm(test_lines[3000:4000:2]):
        tests_str_set = set([' '.join(line) for line in triples_test_reduced])
        flag=False
        count=0
        for train_line in train_lines[30000:60000]:
            trains_str_set = set([' '.join(line) for line in triples_train_reduced])
            if test_line[0] in train_line or test_line[1] in train_line or test_line[2] in train_line:
                if ' '.join(train_line) not in trains_str_set:
                    triples_train_reduced.append(train_line) 
                    flag=True
                    count=count+1
                    if count>=2:
                        break
        if flag==True:
            if ' '.join(test_line) not in tests_str_set:
                triples_test_reduced.append(test_line) 

    print("train_reduced", len(triples_train_reduced),"\n")
    print("train_reduced", triples_train_reduced,"\n")

    print("test_reduced", len(triples_test_reduced),"\n")
    print("test_reduced", triples_test_reduced,"\n")

    positivos=[triple for triple in triples_test_reduced if triple[3]=="1"]
    negativos=[triple for triple in triples_test_reduced if triple[3]=="-1"]
    print("positivos", len(positivos),"\n")
    print("negativos", len(negativos),"\n")

    with open(os.path.join(data_dir, "train_reduced.tsv"), "w") as writer:
        for triple_ in triples_train_reduced:
            writer.write('\t'.join(triple_)+"\n")

    with open(os.path.join(data_dir, "test_reduced.tsv"), "w") as writer:
        for triple_ in triples_test_reduced:
            writer.write('\t'.join(triple_)+"\n")

#createFBReduced(test_start=0,test_end=0,test_step=0,train_start=0,train_end=0,train_step=0)

# Preprocessing-End

In [71]:
def main():

    arg_dict ={
        "task_name": "kg",
        "do_train": False,
        "do_eval": False,
        "do_predict":True,
        "data_dir": data_dir,
        "bert_model": "bert-base-cased",
        "max_seq_length": 200,
        "train_batch_size": 32,
        "eval_batch_size": 512,
        "learning_rate": 5e-5,
        "num_train_epochs": 3.0,
        "output_dir": "./output_FB13/",               
        "gradient_accumulation_steps": 1,
        "seed":42,
        "do_lower_case":False,
        "loss_scale":0,
        "warmup_proportion":0.1
        }

    processors = {
        "kg": KGProcessor,
    }  

    if os.path.exists(arg_dict["output_dir"]) and arg_dict["do_train"]:
        rmtree(arg_dict["output_dir"])
        os.makedirs(arg_dict["output_dir"])

    logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)
    n_gpu=0
    device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
   
    if str(device)=="cuda":
        n_gpu=1
    logger.info("device: {}".format(device))  
    arg_dict["seed"] = random.randint(1, 200)
    random.seed(arg_dict["seed"])
    np.random.seed(arg_dict["seed"])
    torch.manual_seed(arg_dict["seed"])

    if n_gpu > 0:
        torch.cuda.manual_seed_all(arg_dict["seed"])

    task_name = arg_dict["task_name"].lower()

    if task_name not in processors:
        raise ValueError("Task not found: %s" % (task_name))

    processor = processors[task_name]()

    label_list = processor.get_labels(arg_dict["data_dir"])
    num_labels=len(label_list)
    logger.info("num_labels: {}".format(num_labels))   
    logger.info("labels: {}".format(label_list))  
    
    entity_list = processor.get_entities(arg_dict["data_dir"])   
    logger.info("entity_list: {}".format(len(entity_list))) 
    
    relation_list = processor.get_relations(arg_dict["data_dir"])   
    logger.info("relation_list: {}".format(len(relation_list)))  
   
    logger.info("  ") 
    logger.info("//*************Tokenizer-start*************//")   
    tokenizer = BertTokenizer.from_pretrained(arg_dict["bert_model"], do_lower_case=arg_dict["do_lower_case"])
    logger.info("//*************Tokenizer-end*************//")   

    train_examples = None
    num_train_optimization_steps = 0

    if arg_dict["do_train"]:
        train_examples = processor.get_train_examples(arg_dict["data_dir"])

        num_train_optimization_steps = int(
            len(train_examples) / arg_dict["train_batch_size"] / arg_dict["gradient_accumulation_steps"]) * arg_dict["num_train_epochs"]
        logger.info("Num train optimization steps: {}".format(num_train_optimization_steps)) 
    
    logger.info("  ") 
    logger.info("//*************Prepare model-start*************//")   
    model = BertForSequenceClassification.from_pretrained(arg_dict["bert_model"],
              num_labels=num_labels)
    model.to(device) 
    logger.info("//*************Prepare model-end*************//")  

    # Prepare optimizer
    param_optimizer = list(model.named_parameters())
   
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

    optimizer = BertAdam(optimizer_grouped_parameters,
                         lr=arg_dict["learning_rate"],
                         warmup=arg_dict["warmup_proportion"],
                         t_total=num_train_optimization_steps)

    global_step = 0
    nb_tr_steps = 0
    tr_loss = 0

#Training
    if arg_dict["do_train"]: 
        logger.info("  ") 
        logger.info("//*************Running training start*************//") 

        train_features = convert_examples_to_features(
            train_examples, label_list, arg_dict["max_seq_length"], tokenizer)

        logger.info("           Num examples = %d", len(train_examples))
        logger.info("           Batch size = %d", arg_dict["train_batch_size"])
        logger.info("           Num steps = %d", num_train_optimization_steps)

        all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

        train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        train_sampler = RandomSampler(train_data)

        train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=arg_dict["train_batch_size"])

        model.train()

        for epoch in trange(int(arg_dict["num_train_epochs"]), desc="Epoch"):
            tr_loss = 0
            nb_tr_examples, nb_tr_steps = 0, 0
            for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
                batch = tuple(t.to(device) for t in batch)
                input_ids, input_mask, segment_ids, label_ids = batch

                logits = model(input_ids, segment_ids, input_mask, labels=None)


                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits, label_ids)

                loss.backward()

                tr_loss += loss.item()
                nb_tr_examples += input_ids.size(0)
                nb_tr_steps += 1
                if (step + 1) % arg_dict["gradient_accumulation_steps"] == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                    global_step += 1
            logger.info(" Training loss in epoch %s es: %s", epoch+1, tr_loss)
           
        model_to_save = model 

        # If we save using the predefined names, we can load using `from_pretrained`
        output_model_file = os.path.join(arg_dict["output_dir"], WEIGHTS_NAME)
        output_config_file = os.path.join(arg_dict["output_dir"], CONFIG_NAME)

        torch.save(model_to_save.state_dict(), output_model_file)
        model_to_save.config.to_json_file(output_config_file)
        tokenizer.save_vocabulary(arg_dict["output_dir"])
        logger.info("//*************Running training end*************//")

 #Evaluation       
    if os.path.exists(arg_dict["output_dir"]) and arg_dict["do_eval"]: 
        logger.info("      ")
        logger.info("//*************Running evaluation start*************//")      
        model = BertForSequenceClassification.from_pretrained(arg_dict["output_dir"], num_labels=num_labels)
        tokenizer = BertTokenizer.from_pretrained(arg_dict["output_dir"], do_lower_case=arg_dict["do_lower_case"])
        model.to(device)

        eval_examples = processor.get_dev_examples(arg_dict["data_dir"])
        eval_features = convert_examples_to_features(
            eval_examples, label_list, arg_dict["max_seq_length"], tokenizer)
        

        logger.info("       Num examples = %d", len(eval_examples))
        logger.info("       Batch size = %d", arg_dict["eval_batch_size"])

        all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)

        eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        eval_sampler = SequentialSampler(eval_data)
        eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=arg_dict["eval_batch_size"])

        eval_loss = 0
        nb_eval_steps = 0
        preds = []
        model.eval()

        for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader, desc="Evaluating"):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            label_ids = label_ids.to(device)

            with torch.no_grad():
                logits = model(input_ids, segment_ids, input_mask, labels=None)

            loss_fct = CrossEntropyLoss()
            tmp_eval_loss = loss_fct(logits, label_ids)
           
            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1
            
            if len(preds) == 0:
                preds.append(logits.detach().cpu().numpy())
            else:
                preds[0] = np.append(preds[0], logits.detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = preds[0]

        preds_ids = np.argmax(preds, axis=1) 
        result = compute_metrics(task_name, preds_ids, all_label_ids.numpy())


        triples_Evaluation_Report = []
        for cont, label in enumerate(all_label_ids.numpy()):
            triples_Evaluation_Report.append(TripleEvaluationReport(
                    id_triple = cont,
                    id_label = label,
                    id_prediction = preds_ids[cont]
            ))
       
        result['eval_loss'] = eval_loss

        logger.info("***** Eval results *****")
        logger.info("Fecha del reporte "+str(datetime.now()))
        logger.info("Estructura del reporte:  id_triple  label  prediction")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))

        output_eval_file = os.path.join(arg_dict["output_dir"], "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            for triple_ in triples_Evaluation_Report:
                writer.write("%d %d %d\n" % (triple_.id_triple,triple_.id_label, triple_.id_prediction))
        logger.info("//*************Running evaluation end*************//")

#Prediction
    if os.path.exists(arg_dict["output_dir"]) and arg_dict["do_predict"]:
        logger.info("  ")
        logger.info("//*************Running testing start*************//")
        model = BertForSequenceClassification.from_pretrained(arg_dict["output_dir"], num_labels=num_labels)
        tokenizer = BertTokenizer.from_pretrained(arg_dict["output_dir"], do_lower_case=arg_dict["do_lower_case"])
        model.to(device)

        test_examples = processor.get_test_examples(arg_dict["data_dir"])

        test_features = convert_examples_to_features(
            test_examples, label_list, arg_dict["max_seq_length"], tokenizer)

        logger.info("       Num examples = %d", len(test_examples))
        logger.info("       Batch size = %d", arg_dict["eval_batch_size"])

        all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
        all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
        all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)
        all_label_ids = torch.tensor([f.label_id for f in test_features], dtype=torch.long)

  
        test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
        test_sampler = SequentialSampler(test_data)
        test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=arg_dict["eval_batch_size"])
        print(test_dataloader.batch_size)
       
        
        eval_loss = 0
        nb_eval_steps = 0
        preds = []
        model.eval()

        for input_ids, input_mask, segment_ids, label_ids in tqdm(test_dataloader, desc="Testing"):
            input_ids = input_ids.to(device)
            input_mask = input_mask.to(device)
            segment_ids = segment_ids.to(device)
            label_ids = label_ids.to(device)

            with torch.no_grad():
                logits = model(input_ids, segment_ids, input_mask, labels=None)


            loss_fct = CrossEntropyLoss()
            tmp_eval_loss = loss_fct(logits, label_ids)
            
            eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            if len(preds) == 0:
                preds.append(logits.detach().cpu().numpy())        
            else:
                preds[0] = np.append(preds[0], logits.detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        preds = preds[0]

        preds_ids = np.argmax(preds, axis=1)
        result = compute_metrics(task_name, preds_ids,all_label_ids.numpy())

        triples_Evaluation_Report = []
        for cont, label in enumerate(all_label_ids.numpy()):
            triples_Evaluation_Report.append(TripleEvaluationReport(
                    id_triple = cont,
                    id_label = label,
                    id_prediction = preds_ids[cont]
            ))

        result['eval_loss'] = eval_loss

        logger.info("***** Test results *****")
        logger.info("Fecha del reporte "+str(datetime.now()))
        logger.info("Estructura del reporte:  id_triple  label  prediction")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))

        output_test_file = os.path.join(arg_dict["output_dir"], "test_results.txt")
        with open(output_test_file, "w") as writer:           
            for triple_ in triples_Evaluation_Report:
                writer.write("%d %d %d\n" % (triple_.id_triple,triple_.id_label, triple_.id_prediction))
        logger.info("//*************Running testing end*************//")

In [ ]:
if __name__ == "__main__":
    main()